## Notebook to classify snow-covered area (SCA) and determine minimum snow elevations in PlanetScope 4-band images

Rainey Aberle, 2022

### Inputs required:

- __Area of interest (AOI) (`.shp` file):__ can be used to crop images before classifying snow
- __Digital elevation model (DEM) that covers the AOI (`.tif` file):__ used to extract minimum snow elevation in each image
- __PlanetScope images (`.tif` file(s)):__ used to classify snow
- __Trained classifier (`.sav` file):__ used to classify snow in PlanetScope images (find in `planet-snow/inputs-outputs/` or develop in `develop_classifier.ipynb`)

### 1. Import packages

In [4]:
import os
import ee
import geemap
import glob
import numpy as np
import geopandas as gpd
import pandas as pd
import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import show
import matplotlib
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import matplotlib.pyplot as plt
from shapely.geometry import shape
import pickle
import sys
import time

### 2. Define settings and paths in directory

In [2]:
# -----Define site ID (used to load classifier and in output file names)
site_ID = 'WG'

# -----Determine whether to save output figures
save_figures = False # = True to save output figures
crop_to_AOI = True # = True to crop images to AOI before calculating SCA

# -----Define desired EPSG
epsg = 32606

# -----Define paths in directory
# base directory (path to "planet-snow/")
base_path = '/Users/raineyaberle/Research/PhD/planet-snow/'
# image directory
im_path = base_path+'../study-sites/Wolverine/imagery/Planet/adjusted-filtered/'
# figures output directory
figures_out_path = base_path+'../study-sites/Wolverine/figures/SCA/'
# AOI shapefile full path 
AOI_fn = base_path+'../GIS_data/RGI_outlines/Wolverine_RGI.shp'

### 3. GEE Authentication and Initialization

In [3]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

*** Earth Engine *** FINAL DEADLINE: ee.Authenticate will fail after 2022-06-06. Please upgrade. https://developers.google.com/earth-engine/guides/python_install


### 3. Load files from directory and plot

- Area of interest (AOI)
- Digital elevartion model (DEM) from GEE
- PlanetScope images
- Image classifier

In [33]:
# -----Load image names from file
os.chdir(im_path) # change directory
im_names = glob.glob('*.tif') # load all .tif file names
im_names.sort() # sort file names by date

# -----Load image classifier and feature columns
clf_fn = base_path+'inputs-outputs/'+site_ID+'_best_classifier.sav'
clf = pickle.load(open(clf_fn, 'rb'))
feature_cols_fn = base_path+'inputs-outputs/'+site_ID+'_best_classifier_feature_cols.pkl'
feature_cols = pickle.load(open(feature_cols_fn,'rb'))

# -----Add path to functions
sys.path.insert(1, base_path+'functions/')
from classification_utils import crop_images_to_AOI, classify_image, calculate_SCA, determine_min_snow_elev

# -----Load AOI - used for image cropping and DEM filtering
AOI = gpd.read_file(AOI_fn)
# reproject to UTM
AOI_UTM = AOI.to_crs(epsg)
# reformat AOI as ee.Geometry for clipping DEM
AOI_bb_ee = ee.Geometry({"type": "Polygon","coordinates": 
                        [[[AOI.geometry.bounds.minx[0], AOI.geometry.bounds.miny[0]],
                          [AOI.geometry.bounds.maxx[0], AOI.geometry.bounds.miny[0]],
                          [AOI.geometry.bounds.maxx[0], AOI.geometry.bounds.maxy[0]],
                          [AOI.geometry.bounds.minx[0], AOI.geometry.bounds.maxy[0]],
                          [AOI.geometry.bounds.minx[0], AOI.geometry.bounds.miny[0]]]
                        ]})

# -----Query GEE for ArcticDEM, clip to AOI
DEM = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic").clip(AOI_ee)

# -----Plot on GEE map
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(DEM.select('elevation'), {'min': 0, 'max': 1500, 'palette': ['white', 'green']})
Map.centerObject(AOI_ee)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### 4. Classify snow, calculate SCA, determine minimum snow elevation in each image

In [ ]:
# -----Start timer
t1 = time.time()

# -----Initialize image dates and SCA
im_dts = [] # image datetimes
SCA = [] # [m^2]

# -----Crop images if previously selected
if crop_to_AOI==True:
    
    # Crop images if previously selected
    cropped_im_path = crop_images_to_AOI(im_path, im_names, AOI)
    # grab cropped image names
    os.chdir(cropped_im_path) # change directory
    im_names_crop = glob.glob('*_crop.tif')
    im_names_crop.sort() # sort file names by date

    im_names_loop = im_names_crop # im_names to use in loop
    
else:
    
    os.chdir(im_path)
    im_names_loop = im_names # im_names to use in loop
    
# -----Initialize minimum snow and image elevations
snow_elev_min = np.zeros(len(im_names_crop)) # [m] minimum elevation where snow is present
im_elev_min = np.zeros(len(im_names_crop)) # [m] minimum elevation of the image

# -----Loop through images
i=0 # loop counter
for im_name in im_names_loop:

    # extract datetime from image name
    im_dt = np.datetime64(im_name[0:4] + '-' + im_name[4:6] + '-' + im_name[6:8]
                          + ' ' + im_name[9:11] + ':' +im_name[11:13] + ':' + im_name[13:15])
    im_dts = im_dts + [im_dt]

    # open image
    im = rio.open(im_name)

    # classify snow
    plot_output = True
    im_x, im_y, snow, fig = classify_image(im, clf, feature_cols, plot_output)
    fig.suptitle(im_dt)
    plt.show()

    # calculate SCA [m^2]
    SCA = SCA + [calculate_SCA(im, snow)]

    # determine lowest snow elevation
    snow_elev_min[i], im_elev_min[i] = determine_min_snow_elev(DEM, snow, im, im_x, im_y)

    # save figure
    if save_figures==True:
        fig.savefig(figures_out_path+im_name[0:15]+'_SCA.png', dpi=200, facecolor='white', edgecolor='none')
        print('figure saved to file')

    i+=1 # increase loop counter
    
# -----Create pandas.DataFrame for snow cover info
snow_df = pd.DataFrame()
snow_df['datetime'] = im_dts
snow_df['SCA'] = SCA
snow_df['snow_elev_min'] = snow_elev_min
snow_df['im_elev_min'] = im_elev_min

# -----Plot SCA and minimum snow elevation
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8,10))
plt.rcParams.update({'font.size': 14, 'font.serif': 'Arial'})
# SCA
ax1.scatter(snow_df['datetime'], [x/1000 for x in snow_df['SCA']], s=10, color='blue')
ax1.set_ylabel('Snow-covered area [km^2]')
ax1.set_xticklabels([])
ax1.grid()
# min snow elevations (where they equal the min image elev)
ax2.scatter(snow_df.loc[snow_df['snow_elev_min']==snow_df['im_elev_min'], 'datetime'], 
            snow_df.loc[snow_df['snow_elev_min']==snow_df['im_elev_min'], 'snow_elev_min'], 
            s=10, color='orange')
ax2.scatter(snow_df.loc[snow_df['snow_elev_min']!=snow_df['im_elev_min'], 'datetime'], 
            snow_df.loc[snow_df['snow_elev_min']!=snow_df['im_elev_min'], 'snow_elev_min'], 
            s=10, color='blue')
ax2.tick_params(labelrotation=45)
ax2.set_ylabel('Minimum snow elevation [m]')
ax2.grid()
plt.show()

# -----Stop timer
print('Time elapsed: '+str(np.round((time.time()-t1)/60, 2))+' minutes')

### Test method for detecting contours

In [ ]:
# open image
# im_name = im_names_loop[20]
# im = rio.open(im_name)

# # classify snow
# plot_output = True
# im_x, im_y, snow, fig = classify_image(im, clf, feature_cols, plot_output)
# # fig.suptitle(im_dt)
# plt.show()

# # calculate SCA [m^2]
# SCA = SCA + [calculate_SCA(im, snow)]

# # determine lowest snow elevation
# snow_elev_min, im_elev_min = determine_min_snow_elev(DEM, snow, im, im_x, im_y)

# from skimage.measure import find_contours

# # Find contours
# contours = measure.find_contours(snow)
# # Display the image and plot all contours found
# fig, ax = plt.subplots()
# ax.imshow(snow, cmap=plt.cm.gray)
# for contour in contours:
#     ax.plot(contour[:, 1], contour[:, 0], linewidth=1)
# plt.show()